### Scraper Code
##### using Selenium and chromedriver

this code is based upon the work presented in this articale https://medium.com/analytics-vidhya/read-your-network-of-friends-in-facebook-by-scraping-with-python-a012adabb713 by 'Rubén Chuliá Mena', with some changes becuase of the continues changing in Facebook privacy policy and Facebook pages structure

install selenuim

In [1]:
!pip install selenium

install webdriver_manger

In [2]:
!pip install webdriver-manager

In [29]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
#initiate
driver = webdriver.Chrome()

In [30]:
#get the Page
driver.get("https://www.facebook.com/")

In [31]:
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [32]:
#fill The Login fields
user_css_selector = "input[name='email']"
password_css_selector = "input[name='pass']"

username_input = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, user_css_selector))
)
password_input = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, password_css_selector))
)
#replace your userName and your Password by there values
username_input.clear()
username_input.send_keys("your userName")
password_input.clear()
password_input.send_keys("your Password")

In [33]:
#click login button
WebDriverWait(driver, 2).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))
).click()

In [34]:
import time
#wait until page is fully loaded and get the friends list
time.sleep(2)
driver.get("https://www.facebook.com/friends/list")

In [35]:
#using GraphQl to emulate http requests to get the list of friends
import requests

session = requests.session()
session.cookies.update({cookie["name"]: cookie["value"] for cookie in driver.get_cookies()})

In [36]:
import json
import re

pattern = r'\["DTSGInitData",\[\],{"token":"\S+","async_get_token":"\S+?"},\d+\]'
match = re.search(pattern, driver.page_source)
fb_dtsg_token = json.loads(match.group())[2]["token"]

In [37]:
page_info = response_dict["data"]["viewer"]["all_friends"]["page_info"]

In [38]:
import urllib
page_info = dict(has_next_page=True, end_cursor=None)

url = 'https://www.facebook.com/api/graphql/'

headers = {
    "accept": "*/*",
    "accept-language": "es-ES,es;q=0.9",
    "content-type": "application/x-www-form-urlencoded",
    "sec-ch-ua": "\" Not;A Brand\";v=\"99\", \"Google Chrome\";v=\"91\", \"Chromium\";v=\"91\"",
    "sec-ch-ua-mobile": "?0",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "x-fb-friendly-name": "FriendingCometFriendsListPaginationQuery",
    "referrer": "https://www.facebook.com/friends/list",
    "referrerPolicy": "strict-origin-when-cross-origin",
}

friends = []

while page_info["has_next_page"]:
    response = session.post(
        url,
        headers=headers,
        data=urllib.parse.urlencode(
            {
                "fb_dtsg": fb_dtsg_token,
                "fb_api_req_friendly_name": "FriendingCometFriendsListPaginationQuery",
                "variables": json.dumps(
                    {
                        "count": 30,
                        "cursor": page_info["end_cursor"],
                        "scale": 1,
                    }
                ).replace(" ", ""),
                "doc_id": 4268740419836267,
            }
        )
    )
    
    response_dict = json.loads(response.content)
    friend_objects = response_dict["data"]["viewer"]["all_friends"]["edges"]

    friends += [
        dict(
            name=friend['node']['name'].replace(' ','_')
        )
        for friend in friend_objects
        if friend["node"]["__typename"] == "User"
    ]
    
    page_info = response_dict["data"]["viewer"]["all_friends"]["page_info"]
    
print(f"Number of friends: {len(friends)}")

Number of friends: 351


Getting mutual friends section in the code explained in the articale above was used to get mutual friends between profiles in order to get the list of edges, but it does not yield a full list due to facebook privacy policy.

##### explination : 
Q: Why can I no longer see the mutual friends I have with a Friend of mine on Facebook?
A: It's because that mutual friend(s) set his Friends List to be private (Only me) so the total number of mutual friends appear but you only see those who have public Friends List.

Thus I foucsed on a portion of the nodes (100 essential nodes that covers most the clusters in my friends , which are obtained from above) that i could manage to get a List of connection for, and for the messing links i added them manualy (based on my knowldge of the people and thier real-world connection)

In [ ]:
#with open("list_friends_network.txt", "w") as outfile:
#    json.dump(
#        dict(
#            friend_list=friends,
#        ),
#        outfile
#    )